In [1]:
import torch
import echotorch.nn.reservoir as etrs
import echotorch.utils.matrix_generation as mg
from torch.autograd import Variable
from torch.utils.data.dataloader import DataLoader
import numpy as np
import matplotlib.pyplot as plt

# My dataset
from PhyreDataset import PhyreSequentialDataset

In [2]:
# Loading data
train_dataset, test_dataset = PhyreSequentialDataset.train_test_split('data', 0.2)

train_test_split response: test fraction rounded to 0.19985639061752034 (835 simulations)


In [3]:
# Checking if what loaded is indeed what we need
print('input: ', train_dataset.__getitem__(0)[0])
print('output: ', train_dataset.__getitem__(0)[1])

input:  tensor([[0.3500],
        [0.2292],
        [0.0586],
        [0.7500],
        [0.2605],
        [0.1211],
        [0.4141],
        [0.7148],
        [0.0156],
        [0.3500],
        [0.1086],
        [0.0586],
        [0.7500],
        [0.1399],
        [0.1211],
        [0.4141],
        [0.5942],
        [0.0156],
        [0.3500],
        [0.0294],
        [0.0586],
        [0.7500],
        [0.0607],
        [0.1211],
        [0.4141],
        [0.2439],
        [0.0156]])
output:  tensor([[0.4141],
        [0.0362]])


In [4]:
# Setting up input and output dimensions for the network
input_dim = len(train_dataset.__getitem__(0)[0])
output_dim = len(train_dataset.__getitem__(0)[1])

print(input_dim, output_dim)

27 2


In [5]:
# Data loader
trainloader = DataLoader(train_dataset, batch_size=1, shuffle=False, num_workers=2)
testloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=2)

trainloader

In [6]:
# Reservoir hyper-parameters <<<<<------ ASK HOW TO DEFINE THOSE AND WHAT THOSE MEAN
spectral_radius = 1.07
leaky_rate = 0.9261
reservoir_size = 1000
connectivity = 0.1954

In [7]:
# Internal matrix
w_generator = mg.NormalMatrixGenerator(
    connectivity=connectivity,
    spetral_radius=spectral_radius
)

# Input weights
win_generator = mg.NormalMatrixGenerator(
    connectivity=connectivity,
    apply_spectral_radius=False
)

# Bias vector
wbias_generator = mg.NormalMatrixGenerator(
    connectivity=connectivity,
    apply_spectral_radius=False
)

In [8]:
# Create a Leaky-integrated ESN,
# with least-square training algo.
# esn = etrs.ESN(
esn = etrs.LiESN(
    input_dim=input_dim,
    hidden_dim=reservoir_size,
    output_dim=output_dim,
    leaky_rate=leaky_rate,
    learning_algo='inv',
    w_generator=w_generator,
    win_generator=win_generator,
    wbias_generator=wbias_generator,
)

/usr/local/anaconda3/envs/phyre/lib/python3.6/site-packages/echotorch/utils/utility_functions.py:410: UserWarning: torch.eig is deprecated in favor of torch.linalg.eig and will be removed in a future PyTorch release.
torch.linalg.eig returns complex tensors of dtype cfloat or cdouble rather than real tensors mimicking complex tensors.
L, _ = torch.eig(A)
should be replaced with
L_complex = torch.linalg.eigvals(A)
and
L, V = torch.eig(A, eigenvectors=True)
should be replaced with
L_complex, V_complex = torch.linalg.eig(A) (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2894.)
  return torch.max(torch.abs(torch.eig(m)[0])).item()


In [9]:
# For each batch
i = 0
for data in trainloader:
    print('Counter: ', i)
    i+=1
    # Inputs and outputs
    inputs, targets = data

    # Transform data to Variables
    inputs, targets = Variable(inputs), Variable(targets)

    # ESN need inputs and targets
    esn(inputs, targets)

Counter:  0


RuntimeError: size mismatch, got 1000, 1000x27,1